# Objet : Analyse des relations entre champs des données IRVE

## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données (issue TableSchema en cours de validation)
- mise à disposition d'outils de contrôle des relations entre champs des jeux de données tabulaires

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par la détection d'incohérences
- l'analyse a posteriori permet de (re)trouver la logique de dépendance entre les colonnes qui minimise les incohérences

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour les nouvelles données et pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv*"
_________________

# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition, il est à consolider en fonction de l'expertise des concepteurs ( voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb) )

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ associé à une ligne du tableau*

In [3]:
# 'irve' défini en fin de Notebook
mermaid(irve)

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [4]:
from collections import Counter
from observation import Ilist, Iindex
import pandas as pd

chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/'

file = chemin + 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv'
data = pd.read_csv(file, sep=',', low_memory=False)
print('data : ', len(data))

data :  51276


----
## initialisation de l'outil d'analyse
L'outil d'analyse ([voir présentation](https://github.com/loco-philippe/Environmental-Sensing/blob/main/documentation/FR_tabular_structure.ipynb)) est chargé à partir des données Pandas :
- 51 276 lignes
- 50 colonnes
- 211 484 valeurs différentes sur un total de 51 276 x 50 valeurs ("taux d'unicité" de 8.2%)

In [5]:
idxs = Ilist.obj(data)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs), len(idxs.idxlen), sum(idxs.idxlen))

idxs (len, lenlidx, sumcodec) :  51276 50 211370


_________________
# 3 - Analyse des relations entre champs
## Arborescence des champs
- l'examen de "l'arbre de dérivation" indique un faible niveau d'interdépendance entre champs (seuls quelques-uns ne sont pas liés à la racine). 
- Par exemple les champs consolidated_longitude(44) et consolidated_latitude(45) sont bien dérivés du champ coordonneesXY(13)


In [6]:
print(idxs.tree())
# valeur entre parenthèses : nombre de valeurs distinctes

-1: root-derived (51276)
   0 : nom_amenageur (2474)
   1 : siren_amenageur (1182)
   2 : contact_amenageur (548)
   3 : nom_operateur (237)
   4 : contact_operateur (274)
   5 : telephone_operateur (357)
   6 : nom_enseigne (1493)
   7 : id_station_itinerance (18544)
   8 : id_station_local (10731)
   9 : nom_station (15463)
   10: implantation_station (5)
   11: adresse_station (16496)
   12: code_insee_commune (5242)
   13: coordonneesXY (17241)
      44: consolidated_longitude (17069)
      45: consolidated_latitude (17018)
   14: nbre_pdc (50)
   15: id_pdc_itinerance (42457)
   16: id_pdc_local (27186)
   17: puissance_nominale (87)
   18: prise_type_ef (8)
   19: prise_type_2 (8)
   20: prise_type_combo_ccs (8)
   21: prise_type_chademo (7)
   22: prise_type_autre (7)
   23: gratuit (8)
   24: paiement_acte (8)
   25: paiement_cb (7)
   26: paiement_autre (8)
   27: tarification (212)
   28: condition_acces (2)
   29: reservation (7)
   30: horaires (516)
   31: accessibilite_pm

## Identification du champ 'root'
Le champ 'root' est l'identifiant d'une ligne du tableau. Dans le cas présent, aucun champ ne remplit ce rôle. Le champ le plus proche est le champ 'id_pdc_itinerance' avec 42 457 valeurs pour 51 276 lignes.

In [7]:
champ = idxs.nindex
root = Iindex.ext(list(range(len(idxs))))
id_pdc_itinerance = champ('id_pdc_itinerance')
ecart = id_pdc_itinerance.coupling(root)
print('lignes en écart : ',len(ecart))
c = Counter(id_pdc_itinerance.codec).most_common(5)
id_pdc_itinerance.tostdcodec(full=False)
print('les 5 pdc avec le plus de lignes : \n', c, '\n')

lignes en écart :  16642
les 5 pdc avec le plus de lignes : 
 [('Non concerné', 519), ('FR55CP84000', 50), ('FR55CP92140', 29), ('FR55CP78140', 28), ('FR55CP83310', 23)] 



- le principal écart est lié à la présence de l'historique des modifications.
- l'écart résiduel (1068 lignes) est à traiter de façon spécifique -> cf exemples d'écarts ci-dessous

In [8]:
last_modified = champ('last_modified')
id_pdc = Iindex.merging([id_pdc_itinerance, last_modified], 'id_pdc')
ecart = id_pdc.coupling(root)
print('lignes en écart : ',len(ecart))
c = Counter(id_pdc.codec).most_common(5)
id_pdc.tostdcodec(full=False)
print('les 5 pdc avec le plus de lignes : \n', c, '\n')

lignes en écart :  1068
les 5 pdc avec le plus de lignes : 
 [(('FR55CP84000', '2023-03-03T17:39:50.555000'), 50), (('Non concerné', '2023-03-03T17:37:07.612000'), 34), (('FR55CP92140', '2023-03-03T17:39:50.555000'), 29), (('FR55CP78140', '2023-03-03T17:39:50.555000'), 28), (('FR55CP83310', '2023-03-03T17:39:50.555000'), 23)] 



In [9]:
data.loc[ecart,['id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY', 'nbre_pdc', 'last_modified']]

,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY,nbre_pdc,last_modified
47568,Non concerné,Non concerné,Parking L'Aérodrome de la Baie de Somme,"[1.8255189491394663, 50.13950991485836]",3,2023-03-03T17:35:35.997000
47569,Non concerné,Non concerné,Parking L'Aérodrome de la Baie de Somme,"[1.8255189491394663, 50.13950991485836]",3,2023-03-03T17:35:35.997000
47567,Non concerné,Non concerné,Parking L'Aérodrome de la Baie de Somme,"[1.8255189491394663, 50.13950991485836]",3,2023-03-03T17:35:35.997000
2518,FR55CP92140,FR55CP92140,AVENUE JEAN JAURES - CLAMART,"[2.267834, 48.808019]",1,2023-03-03T17:39:50.555000
2388,FR55CP92140,FR55CP92140,POMPIDOU - CLAMART,"[2.23637, 48.784747]",1,2023-03-03T17:39:50.555000
...,...,...,...,...,...,...
41150,FRS49E492230022,FRS49P49223A,OuestCharge - Diva Sp - Murs-Erigne - De Grand...,"[-0.528586, 47.396864]",2,2023-03-03T17:36:58.803000
9077,FRP07E6748200322,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",41,2023-03-03T17:39:32.426000
9106,FRP07E6748200322,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",1,2023-03-03T17:39:32.426000
4357,FRP07E674820038,FRP07P67482003,STRASBOURG - Wodli,"[7.73762, 48.587002]",41,2023-03-03T17:39:32.426000


- l'élimination des doublons (en conservant la date de modification la plus résente) permet d'avoir une ligne par point de charge.

In [10]:
data2 = data.sort_values(by='last_modified').drop_duplicates('id_pdc_itinerance', keep='last').reset_index(drop=True)
print('data2 : ', len(data2))

data2 :  42457


In [11]:
idxs = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs), len(idxs.idxlen), sum(idxs.idxlen))

idxs (len, lenlidx, sumcodec) :  42457 50 193355


In [12]:
champ = idxs.nindex
root = Iindex.ext(list(range(len(idxs))))
print("Couplage entre le champ 'id_pdc_itinerance' et la liste des lignes : ", champ('id_pdc_itinerance').iscoupled(root))

Couplage entre le champ 'id_pdc_itinerance' et la liste des lignes :  True


## Représentation des stations

- deux champs sont candidats pour identifier les stations : 'id_station_itinerance', 'nom_station'
- Le champ le plus approprié est 'id_station_itinerance' (plus grand nombre de valeurs différentes)
- une autre solution pourrait être de considérer qu'une station est identifiée par le couple 'id_station_itinerance', 'nom_station' mais ceci conduit à doublonner une station pour laquelle une erreur de nom a été commise (cf exemple ci-dessous).
- les exemples ci-dessous montrent les incohérences de nom de station (288 lignes concernées)

In [13]:
id_station_itinerance = champ('id_station_itinerance')
nom_station = champ('nom_station')
print('le nom dérive de la station : ', id_station_itinerance.isderived(nom_station), '\n')

ecart_nom = id_station_itinerance.coupling(nom_station)
c_nom = Counter(id_station_itinerance.codec).most_common(5)
id_station_itinerance.reindex()

print('nombre de lignes en écart : ',len(ecart_nom))
print('les 5 stations avec le plus de noms : \n', c_nom, '\n')

le nom dérive de la station :  False 

nombre de lignes en écart :  288
les 5 stations avec le plus de noms : 
 [('Non concerné', 30), ('FRH14P59307001', 2), ('FRFR1EAYRU', 2), ('FRS56PVCVTUT', 2), ('FRH14P59578001', 2)] 



In [14]:
print('stations avec plusieurs noms : \n')
data2.loc[ecart_nom,['id_pdc_itinerance', 'id_station_itinerance', 'nom_station', 'coordonneesXY']]

stations avec plusieurs noms : 



,id_pdc_itinerance,id_station_itinerance,nom_station,coordonneesXY
21833,FRH14P59307001,FRH14P59307001,Holque - Route de l'église,"[2.2037, 50.85458]"
37701,FRH14E593070011,FRH14P59307001,HOLQUE - Route de l'église,"[2.2037, 50.85458]"
33280,FRFR1EAYRU2,FRFR1EAYRU,"Loos, Parking de la Gare","[3.02,50.61]"
33281,FRFR1EAYRU1,FRFR1EAYRU,"Loos, Parking de la Gare","[3.02,50.61]"
33282,FRFR1EAYRU3,FRFR1EAYRU,"Loos, Parking de la Gare","[3.02,50.61]"
...,...,...,...,...
19831,FRLE2ESTTM2,FRLE2PZYW3BNYUDP,Leclerc Mios,"[-0.918739, 44.618314]"
19832,FRLE2ESTTM1,FRLE2PZYW3BNYUDP,Leclerc Mios,"[-0.918739, 44.618314]"
21830,FRH14P59338001,FRH14P59338001,Ledringhem - Parking du souvenir,"[2.439028, 50.854477]"
34365,FRH14E593380012,FRH14P59338001,LEDRINGHEM - Route D'Arneke,"[2.439028, 50.854477]"


## localisation des stations
Les stations sont localisées au travers d'une coordonnée géographique et une adresse complète cette localisation. De même que pour la dénomination des stations, deux questions peuvent être posées :
- peut-on avoir plusieurs stations avec la même coordonnée ?
- peut-on avoir plusieurs coordonnées pour une même station ?

L'analyse ci-dessous montre que des erreurs d'identification de stations conduisent à répondre oui à ces deux questions.

In [15]:
champ = idxs.nindex
coordonneesXY = champ('coordonneesXY')

ecart_XY = id_station_itinerance.coupling(coordonneesXY)
c_XY = Counter(id_station_itinerance.codec).most_common(5)
id_station_itinerance.reindex()

print('nombre de lignes en écart : ',len(ecart_XY))

print('les 5 stations avec le plus de positions différentes : \n', c_XY, '\n')

nombre de lignes en écart :  705
les 5 stations avec le plus de positions différentes : 
 [('Non concerné', 28), ('FR073PCAMAIEUFR', 8), ('FRFR1PEHNKRRPQNR', 5), ('FRFR1PUXIVU5GSXH', 5), ('FRFR1PD0AIJIWGHD', 4)] 



In [16]:
print('stations avec plusieurs coordonnées : \n')
data2.loc[ecart_XY,['id_station_itinerance', 'nom_station', 'coordonneesXY']]

stations avec plusieurs coordonnées : 



,id_station_itinerance,nom_station,coordonneesXY
29843,FRFR1PDZVOZRKVZZ,"Castelnaudary, Avenue des pyrénées - 3661","[1.95471108, 43.30053645]"
29856,FRFR1PDZVOZRKVZZ,"Castelnaudary, Avenue des pyrénées - 3661","[1.95471108, 43.30053645]"
29901,FRFR1PDZVOZRKVZZ,"Castelnaudary, Avenue des pyrénées - 3661","[3.513644, 43.651538]"
29907,FRFR1PDZVOZRKVZZ,"Castelnaudary, Avenue des pyrénées - 3661","[3.03493, 43.325232]"
41109,Non concerné,LIDL ST PIERRE SUR DIVES,"[-0.0373753,49.020012]"
...,...,...,...
32704,FRFR1PRPDMF5LYE0,"Villefranche-sur-Saône, 733 Av. Théodore Braun","[4.7394, 45.97827]"
32701,FRFR1PRPDMF5LYE0,"Villefranche-sur-Saône, 733 Av. Théodore Braun","[4.738635, 45.978188]"
32702,FRFR1PRPDMF5LYE0,"Villefranche-sur-Saône, 733 Av. Théodore Braun","[4.738635, 45.978188]"
12046,FREBNPIWGHRPVBV0,"Roche-La-Moliere, Parking stade Beaulieu","[4.31678503, 45.42282428]"


In [17]:
ecart_stat = coordonneesXY.coupling(id_station_itinerance)
c_stat = Counter(coordonneesXY.codec).most_common(5)
coordonneesXY.reindex()

print('lignes en écart : ',len(ecart_stat))

print('les 5 positions avec le plus grand nombre de stations associées : \n', c_stat, '\n')

lignes en écart :  7764
les 5 positions avec le plus grand nombre de stations associées : 
 [('[2.322582, 48.837734]', 60), ('[2.298571, 48.896523]', 40), ('[2.477817, 48.882166]', 22), ('[2.523685, 48.9908]', 20), ('[2.21,48.78]', 20)] 



In [18]:
print('positions avec plusieurs stations associées : \n')
data2.loc[ecart_stat,['coordonneesXY', 'id_station_itinerance', 'nom_station']]

positions avec plusieurs stations associées : 



,coordonneesXY,id_station_itinerance,nom_station
19956,"[7.021575, 43.575086]",FRA16PWIIZ10611,LE CANNET PARKING VAL DAZUR
19955,"[7.021575, 43.575086]",FRA16PWIIZ10621,LE CANNET PARKING VAL DAZUR
3768,"[-1.913791, 48.675262]",FRS35PSD357812,SAINT COULOMB - PLACE DU MARCHE
3736,"[-1.913791, 48.675262]",FRS35PSD357811,SAINT COULOMB - PLACE DU MARCHE
40074,"[0.319564, 49.320563]",FRLMSP1000118147,SANEF BEUZEVILLE SUD
...,...,...,...
10549,"[2.399729, 48.814589]",FRSIGPSIGE26912,SIGEIF - 23 BOULEVARD DE BRANDEBOURG - IVRY-S...
20128,"[7.015534, 43.609589]",FRA16PWIIZ4711,MOUGINS PARKING SALLE OMNISPORTS DU FONT DE LORME
20129,"[7.015534, 43.609589]",FRA16PWIIZ4712,MOUGINS PARKING SALLE OMNISPORTS DU FONT DE LORME
4782,"[0.090439, 44.313465]",FRS47PMB478712,CASTELJALOUX - PLACE DE LEGLISE


-----------------------
# Annexe : Initialisation du modèle 'mermaid'

In [1]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mermaid(graph):
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  display(Image(url="https://mermaid.ink/img/" + base64_string))

In [2]:
# modèle de données suivant l'application 'mermaid' 
irve = """ 
erDiagram
    AMENAGEUR ||..o{ STATION : amenage
    AMENAGEUR {
        string nom_amenageur
        string siren_amenageur
        string contact_amenageur 
    }
    OPERATEUR ||..|{ STATION : "exploite pour le compte d un amenageur"
    OPERATEUR {
        string contact_operateur PK "M"
        string nom_enseigne "M" 
        string nom_operateur 
        string telephone_operateur 
    }
    STATION {
        string  id_station_itinerance PK "M"
        string  nom_station "M"
        enum    implantation_station "M"
        integer nbre_pdc "M"
        string  condition_acces "M"
        string  horaires "M"
        boolean station_deux_roues "M"
        date    date_maj "M"
        string  id_station_local
        enum    raccordement
        string  num_pdl
        date    date_mise_en_service 
    }
    LOCALISATION ||--o{ STATION : "se situe"
    LOCALISATION {
       array   coordonneesXY PK "M"
       string  adresse_station "M"
       string  code_insee_commune 
    }
    STATION ||--o{ POINT_DE_CHARGE : regroupe
    POINT_DE_CHARGE {
        string id_pdc_itinerance PK "M Root"
        number puissance_nominale "M"
        boolean prise_type_ef "M"
        boolean prise_type_2 "M"
        boolean prise_type_2combo_ccs "M"
        boolean prise_type_chademo "M"
        boolean prise_type_autre "M"
        boolean paiement_acte "M"
        boolean paiement_autre "M"
        boolean reservation "M"
        enum    accessibilite_pmr "M"
        string  restriction_gabarit "M"
        string id_pdc_local
        boolean gratuit
        boolean paiement_cb
        string  tarification
        string  observations
        boolean cable_t2_attache 
    }
"""